In [53]:
import pandas as pd

test = pd.read_csv('test.csv')
a = pd.read_csv('FINAL_CHECKING.csv')
a = a[['CI_HOUR']]
#a = a.rename(columns={'CI_HOUR': 'CI_HOUR_NEW'})
test = test[['ARI_CO','ARI_PO','ATA','U_WIND', 'V_WIND','AIR_TEMPERATURE','BN']]
test = pd.concat([test,a],axis=1)

In [54]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test['ATA'] = pd.to_datetime(test['ATA'])

train_df['ATA'] = pd.to_datetime(train_df['ATA'])
train_df = train_df.sort_values(by='ATA')

mapping_dict_train = {}
for _, row in train_df.iterrows():
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'])
    
    if any(pd.isna(val) for val in key):
        continue
    
    mapping_dict_train[key] = (row['ATA'], row['CI_HOUR'])

# 4. 후처리 함수 정의 및 적용
def compute_post_processed_target_v11(row):
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'])
    
    # 해당 샘플의 날짜보다 미래의 train_df의 데이터 중, 키가 일치하는 데이터의 날짜 참조
    if key in mapping_dict_train and mapping_dict_train[key][0] > row['ATA']:
        time_difference = (mapping_dict_train[key][0] - row['ATA']).total_seconds() / 3600
        # 시간 차이가 70일(1680시간) 이상 나는 경우 원래 값 반환
        if time_difference >= 2160: #1920, 1440, 1200, 1680
            return row['CI_HOUR']
        # 해당 Train 샘플의 CI_HOUR 값과 시간 차이를 합산하여 반환
        return abs(time_difference) + mapping_dict_train[key][1]
    
    # 일치하는 데이터가 없는 경우, 현재 값 반환
    return row['CI_HOUR']

# 함수 적용 및 결과 확인
test['Post_Processed_CI_HOUR_v11'] = test.apply(compute_post_processed_target_v11, axis=1)
print(test[['ATA', 'CI_HOUR', 'Post_Processed_CI_HOUR_v11']])

In [ ]:
import pandas as pd

# 1. train_df 데이터 로드
train_df = pd.read_csv("train.csv")
test['ATA'] = pd.to_datetime(test['ATA'])

# 2. train_df의 'ATA' 열을 datetime 형식으로 변환
train_df['ATA'] = pd.to_datetime(train_df['ATA'])

# 3. train_df에서 각 ARI_PO 별 CI_HOUR의 최대값 계산
max_values = train_df.groupby('ARI_PO')['CI_HOUR'].max()

# 4. test 데이터프레임의 각 행에 대해 최대값과 비교하여 대체
def cap_by_max(row):
    ari_po = row['ARI_PO']
    if ari_po in max_values:
        return min(row['Post_Processed_CI_HOUR_v11'], max_values[ari_po])
    return row['Post_Processed_CI_HOUR_v11']

test['Adjusted_CI_HOUR'] = test.apply(cap_by_max, axis=1)

# 결과 확인
print(test[['ARI_PO', 'Post_Processed_CI_HOUR_v11', 'Adjusted_CI_HOUR']])

       ARI_PO  Post_Processed_CI_HOUR_v11  Adjusted_CI_HOUR
0        GIW5                    1.082778          1.082778
1        WEY7                   10.325580         10.325580
2        NGG6                   44.690556         44.690556
3        FFM2                   75.187859         75.187859
4        QYY1                    0.000000          0.000000
...       ...                         ...               ...
220486   TMW2                   96.477516         96.477516
220487   GRQ5                   56.346157         56.346157
220488   VYJ1                   13.503333         13.503333
220489   QQW1                    0.000000          0.000000
220490   PAF4                   46.912231         46.912231

[220491 rows x 3 columns]


In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['CI_HOUR'] = test['Adjusted_CI_HOUR']
test = pd.read_csv('test.csv')
all = pd.concat([test,sub],axis=1)
#all['CI_HOUR'][all['DIST'] == 0] = 0
all.loc[all['DIST'] == 0, 'CI_HOUR'] = 0
sub['CI_HOUR'] = all['CI_HOUR']
#sub['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
sub.to_csv("pb2160_diff2_2880.csv",index=False)
sub

,SAMPLE_ID,CI_HOUR
0,TEST_000000,1.082778
1,TEST_000001,10.325580
2,TEST_000002,44.690556
3,TEST_000003,75.187859
4,TEST_000004,0.000000
...,...,...
220486,TEST_220486,96.477516
220487,TEST_220487,56.346157
220488,TEST_220488,13.503333
220489,TEST_220489,0.000000


In [45]:
sub.describe()

,CI_HOUR
count,220491.000000
mean,46.526664
std,122.069959
min,0.000000
25%,0.000000
50%,13.346495
75%,51.820307
max,2159.130556
